In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import ipywidgets as widgets
from ipywidgets import interact

# --- 1. Carregar e Preparar os Dados ---
try:
    # Usando os caminhos que você confirmou
    df_cases = pd.read_excel('/content/drive/MyDrive/dados_covid/casos_confirmadoXobitos.xlsx', engine='openpyxl')
    df_vax = pd.read_excel('/content/drive/MyDrive/dados_covid/96ab09e2-0583-4e0a-bedf-d7533e8c5c2e.xlsx', engine='openpyxl')

    # --- Preparação dos DataFrames ---
    df_cases['DATA AJUSTADA'] = pd.to_datetime(df_cases['DATA AJUSTADA'])
    df_cases.set_index('DATA AJUSTADA', inplace=True)
    df_vax['Data da Vacina'] = pd.to_datetime(df_vax['Data da Vacina'])
    df_vax.set_index('Data da Vacina', inplace=True)
    df_vax.rename(columns={'Total de Doses Aplicadas': 'Doses_Aplicadas'}, inplace=True)

    # --- Junção dos DataFrames ---
    df_combined = df_cases.join(df_vax, how='outer')
    df_combined.fillna(0, inplace=True)
    df_combined = df_combined.astype({'Confirmados': 'int64', 'Óbitos': 'int64', 'Doses_Aplicadas': 'int64'})
    df_combined.sort_index(inplace=True)


    # --- 3. Função para Criar o Gráfico Combinado ---
    def plot_combined_data(year):
        df_year = df_combined[df_combined.index.year == year]
        if df_year.empty:
            print(f"Não há dados disponíveis para o ano de {year}.")
            return

        fig, ax1 = plt.subplots(figsize=(17, 8))
        fig.suptitle(f'COVID-19: Casos, Óbitos e Vacinação em MS ({year})', fontsize=18)

        # --- Plotagem das linhas ---
        color1 = 'dodgerblue'
        ax1.set_xlabel('Data')
        ax1.set_ylabel('Número de Casos / Óbitos', color=color1, fontsize=12)
        line1 = ax1.plot(df_year.index, df_year['Confirmados'], color=color1, label='Casos Confirmados', alpha=0.8)
        line2 = ax1.plot(df_year.index, df_year['Óbitos'], color='crimson', label='Óbitos', linestyle=':')
        ax1.tick_params(axis='y', labelcolor=color1)
        ax1.grid(True, axis='y', linestyle='--', alpha=0.6)

        ax2 = ax1.twinx()
        color2 = 'green'
        ax2.set_ylabel('Doses de Vacina Aplicadas', color=color2, fontsize=12)
        line3 = ax2.plot(df_year.index, df_year['Doses_Aplicadas'], color=color2, label='Doses Aplicadas', alpha=0.7)
        ax2.tick_params(axis='y', labelcolor=color2)
        ax2.set_ylim(bottom=0)

        # --- LÓGICA DE ANOTAÇÃO COM POSICIONAMENTO CENTRALIZADO ---
        for month_num in range(1, 13):
            month_data = df_year[df_year.index.month == month_num]
            if not month_data.empty:
                # Posição X será sempre o dia 15 do mês
                x_position = pd.Timestamp(year=year, month=month_num, day=15)

                # Anotações para Casos
                total_cases = month_data['Confirmados'].sum()
                if total_cases > 0:
                    peak_value = month_data['Confirmados'].max()
                    ax1.annotate(f'{total_cases:,}', xy=(x_position, peak_value),
                                 xytext=(0, 15), textcoords='offset points',
                                 ha='center', fontsize=9, color='darkblue', fontweight='bold')

                # Anotações para Óbitos
                total_deaths = month_data['Óbitos'].sum()
                if total_deaths > 0:
                    peak_value = month_data['Óbitos'].max()
                    ax1.annotate(f'{total_deaths}', xy=(x_position, peak_value),
                                 xytext=(0, 5), textcoords='offset points',
                                 ha='center', fontsize=9, color='darkred', fontweight='bold')

                # Anotações para Doses
                total_vax = month_data['Doses_Aplicadas'].sum()
                if total_vax > 0:
                    peak_value = month_data['Doses_Aplicadas'].max()
                    ax2.annotate(f'{total_vax:,}', xy=(x_position, peak_value),
                                 xytext=(0, 5), textcoords='offset points',
                                 ha='center', fontsize=9, color='darkgreen', fontweight='bold')

        # --- Formatação final ---
        ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b/%Y'))
        ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
        start_date = pd.Timestamp(f'{year}-01-01')
        end_date = pd.Timestamp(f'{year}-12-31')
        ax1.set_xlim(start_date, end_date)
        fig.autofmt_xdate()
        lines = line1 + line2 + line3
        labels = [l.get_label() for l in lines]
        ax1.legend(lines, labels, loc='upper left', fontsize=10)
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.show()

    # --- Widget Interativo ---
    years_available = sorted(df_combined.index.year.unique())
    interact(plot_combined_data, year=widgets.Dropdown(
        options=years_available,
        value=years_available[0],
        description='Selecione o Ano:'
    ))

except FileNotFoundError as e:
    print(f"Erro: Arquivo não encontrado. {e.filename}")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")

interactive(children=(Dropdown(description='Selecione o Ano:', options=(2020, 2021, 2022, 2023, 2024, 2025), v…